In [2]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span

In [4]:
nlp = spacy.load('en_core_web_sm')

/home/mangy007/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 176 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [26]:
text = 'Usain St Leo Bolt, OJ, CD is a Jamaican former sprinter. He is a world record holder in the 100 metres, 200 metres and 4 × 100 metres relay. Owing to his achievements and dominance in sprint competition, he is widely considered to be the greatest sprinter of all time.'

In [27]:
doc = nlp(text)

In [19]:
def get_entities(sent):
    ent1 = ''
    ent2 = ''

    prev_tok_dep = ''
    prev_tok_text = ''

    prefix = ''
    modifier = ''
    
    for token in sent:
        if token.dep_ != 'punct':
            if token.dep_ != 'compound':
                prefix = token.text
                if prev_tok_dep == "compound":
                    prefix = prev_tok_text + " "+ token.text
            if token.dep_.endswith("mod") == True:
                modifier = token.text
                if prev_tok_dep == "compound":
                    modifier = prev_tok_text + " "+ token.text
            if token.dep_.find("subj") == True:
                ent1 = modifier +" "+ prefix + " "+ token.text
                prefix = ""
                modifier = ""
                prev_tok_dep = ""
                prev_tok_text = ""
            if token.dep_.find("obj") == True:
                ent2 = modifier +" "+ prefix +" "+ token.text

            prev_tok_dep = token.dep_
            prev_tok_text = token.text
            
    return [ent1.strip(), ent2.strip()]

In [29]:
for sent in doc.sents:
    print(get_entities(sent))

['CD CD', '']
['He He', '100 metres metres']
['he he', 'greatest time time']


In [30]:
for sent in doc.sents:
    for token in sent:
        print(token.text,'-',token.dep_)

Usain - compound
St - compound
Leo - compound
Bolt - nsubj
, - punct
OJ - appos
, - punct
CD - nsubj
is - ROOT
a - det
Jamaican - amod
former - amod
sprinter - attr
. - punct
He - nsubj
is - ROOT
a - det
world - compound
record - compound
holder - attr
in - prep
the - det
100 - nummod
metres - pobj
, - punct
200 - nummod
metres - conj
and - cc
4 - quantmod
× - compound
100 - nummod
metres - compound
relay - conj
. - punct
Owing - advcl
to - prep
his - poss
achievements - pobj
and - cc
dominance - conj
in - prep
sprint - compound
competition - pobj
, - punct
he - nsubjpass
is - auxpass
widely - advmod
considered - ROOT
to - aux
be - xcomp
the - det
greatest - amod
sprinter - attr
of - prep
all - det
time - pobj
. - punct
